Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/reinforcement-learning/minecraft-on-distributed-compute/minecraft.png)

# Reinforcement Learning in Azure Machine Learning - Training a Minecraft agent using custom environments

This tutorial will show how to set up a more complex reinforcement
learning (RL) training scenario.  It demonstrates how to train an agent to
navigate through a lava maze in the Minecraft game using Azure Machine
Learning.

**Please note:** This notebook trains an agent on a randomly generated
Minecraft level.  As a result, on rare occasions, a training run may fail
to produce a model that can solve the maze.  If this happens, you can
re-run the training step as indicated below.

**Please note:** This notebook uses 1 NC6 type node and 8 D2 type nodes
for up to 5 hours of training, which corresponds to approximately $9.06 (USD)
as of May 2020.

Minecraft is currently one of the most popular video
games and as such has been a study object for RL.  [Project 
Malmo](https://www.microsoft.com/en-us/research/project/project-malmo/) is
a platform for artificial intelligence experimentation and research built on
top of Minecraft.  We will use Minecraft [gym](https://gym.openai.com) environments from Project
Malmo's 2019 MineRL competition, which are part of the 
[MineRL](http://minerl.io/docs/index.html) Python package.

Minecraft environments require a display to run, so we will demonstrate
how to set up a virtual display within the docker container used for training.
Learning will be based on the agent's visual observations.  To
generate the necessary amount of sample data, we will run several
instances of the Minecraft game in parallel.  Below, you can see a video of
a trained agent navigating a lava maze. Starting from the green position,
it moves to the blue position by moving forward, turning left or turning right:

<table style="width:50%">
  <tr>
      <th style="text-align: center;">
          <img src="./images/lava_maze_minecraft.gif" alt="Minecraft lava maze" align="middle" margin-left="auto" margin-right="auto"/>
      </th>
  </tr>
  <tr style="text-align: center;">
      <th>Fig 1. Video of a trained Minecraft agent navigating a lava maze.</th>
  </tr>
</table>

The tutorial will cover the following steps:
- Initializing Azure Machine Learning resources for training
- Training the RL agent with Azure Machine Learning service
- Monitoring training progress
- Reviewing training results


## Prerequisites

The user should have completed the Azure Machine Learning introductory tutorial.
You will need to make sure that you have a valid subscription id, a resource group and a
workspace.  For detailed instructions see [Tutorial: Get started creating
your first ML experiment.](https://docs.microsoft.com/en-us/azure/machine-learning/tutorial-1st-experiment-sdk-setup)

In addition, please follow the instructions in the [Reinforcement Learning in
Azure Machine Learning - Setting Up Development Environment](../setup/devenv_setup.ipynb)
notebook to correctly set up a Virtual Network which is required for completing 
this tutorial.

While this is a standalone notebook, we highly recommend going over the
introductory notebooks for RL first.
- Getting started:
  - [RL using a compute instance with Azure Machine Learning service](../cartpole-on-compute-instance/cartpole_ci.ipynb)
  - [Using Azure Machine Learning compute](../cartpole-on-single-compute/cartpole_sc.ipynb)
- [Scaling RL training runs with Azure Machine Learning service](../atari-on-distributed-compute/pong_rllib.ipynb)


## Initialize resources

All required Azure Machine Learning service resources for this tutorial can be set up from Jupyter.
This includes:
- Connecting to your existing Azure Machine Learning workspace.
- Creating an experiment to track runs.
- Creating remote compute targets for [Ray](https://docs.ray.io/en/latest/index.html).

### Azure Machine Learning SDK

Display the Azure Machine Learning SDK version.

In [ ]:
import azureml.core
print("Azure Machine Learning SDK Version: ", azureml.core.VERSION) 

### Connect to workspace

Get a reference to an existing Azure Machine Learning workspace.

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, sep=' | ')

### Create an experiment

Create an experiment to track the runs in your workspace.  A
workspace can have multiple experiments and each experiment
can be used to track multiple runs (see [documentation](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.experiment.experiment?view=azure-ml-py)
for details).

In [ ]:
from azureml.core import Experiment

exp = Experiment(workspace=ws, name='minecraft-maze')

### Create or attach an existing compute resource

A compute target is a designated compute resource where you
run your training script.  For more information, see [What
are compute targets in Azure Machine Learning service?](https://docs.microsoft.com/en-us/azure/machine-learning/concept-compute-target).

#### GPU target for Ray head

In the experiment setup for this tutorial, the Ray head node
will run on a GPU-enabled node.  A maximum cluster size
of 1 node is therefore sufficient.  If you wish to run
multiple experiments in parallel using the same GPU
cluster, you may elect to increase this number.  The cluster
will automatically scale down to 0 nodes when no training jobs
are scheduled (see `min_nodes`).

The code below creates a compute cluster of GPU-enabled NC6
nodes.  If the cluster with the specified name is already in
your workspace the code will skip the creation process.

Note that we must specify a Virtual Network during compute
creation to allow communication between the cluster running
the Ray head node and the additional Ray compute nodes.  For
details on how to setup the Virtual Network, please follow the
instructions in the "Prerequisites" section above.

**Note: Creation of a compute resource can take several minutes**

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# please enter the name of your Virtual Network (see Prerequisites -> Workspace setup)
vnet_name = 'your_vnet'

# name of the Virtual Network subnet ('default' the default name)
subnet_name = 'default'

gpu_cluster_name = 'gpu-cl-nc6-vnet'

try:
    gpu_cluster = ComputeTarget(workspace=ws, name=gpu_cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(
        vm_size='Standard_NC6',
        min_nodes=0,
        max_nodes=1,
        vnet_resourcegroup_name=ws.resource_group,
        vnet_name=vnet_name,
        subnet_name=subnet_name)

    gpu_cluster = ComputeTarget.create(ws, gpu_cluster_name, compute_config)
    gpu_cluster.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

    print('Cluster created.')

#### CPU target for additional Ray nodes

The code below creates a compute cluster of D2 nodes. If the cluster with the specified name is already in your workspace the code will skip the creation process.

This cluster will be used to start additional Ray nodes
increasing the clusters CPU resources.

**Note: Creation of a compute resource can take several minutes**

In [ ]:
cpu_cluster_name = 'cpu-cl-d2-vnet'

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(
        vm_size='STANDARD_D2',
        min_nodes=0,
        max_nodes=10,
        vnet_resourcegroup_name=ws.resource_group,
        vnet_name=vnet_name,
        subnet_name=subnet_name)

    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    cpu_cluster.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

    print('Cluster created.')

## Training the agent

### Training environments

This tutorial uses custom docker images (CPU and GPU respectively)
with the necessary software installed.  The
[Environment](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments)
class stores the configuration for the training environment.  The docker
image is set via `env.docker.base_image` which can point to any
publicly available docker image.  `user_managed_dependencies`
is set so that the preinstalled Python packages in the image are preserved.

Note that since Minecraft requires a display to start, we set the `interpreter_path`
such that the Python process is started via **xvfb-run**.

In [ ]:
import os
from azureml.core import Environment

max_train_time = os.environ.get("AML_MAX_TRAIN_TIME_SECONDS", 5 * 60 * 60)

def create_env(env_type):
    env = Environment(name='minecraft-{env_type}'.format(env_type=env_type))

    env.docker.enabled = True
    env.docker.base_image = 'akdmsft/minecraft-{env_type}'.format(env_type=env_type)

    env.python.interpreter_path = "xvfb-run -s '-screen 0 640x480x16 -ac +extension GLX +render' python"
    env.environment_variables["AML_MAX_TRAIN_TIME_SECONDS"] = str(max_train_time)
    env.python.user_managed_dependencies = True
    
    return env
    
cpu_minecraft_env = create_env('cpu')
gpu_minecraft_env = create_env('gpu')

### Training script

As described above, we use the MineRL Python package to launch
Minecraft game instances.  MineRL provides several OpenAI gym
environments for different scenarios, such as chopping wood.
Besides predefined environments, MineRL lets its users create
custom Minecraft environments through
[minerl.env](http://minerl.io/docs/api/env.html).  In the helper
file **minecraft_environment.py** provided with this tutorial, we use the
latter option to customize a Minecraft level with a lava maze
that the agent has to navigate. The agent receives a negative
reward of -1 for falling into the lava, a negative reward of
-0.02 for sending a command (i.e. navigating through the maze
with fewer actions yields a higher total reward) and a positive reward
of 1 for reaching the goal.  To encourage the agent to explore
the maze, it also receives a positive reward of 0.1 for visiting
a tile for the first time.

The agent learns purely from visual observations and the image
is scaled to an 84x84 format, stacking four frames.  For the
purposes of this example, we use a small action space of size
three: move forward, turn 90 degrees to the left, and turn 90
degrees to the right.

The training script itself registers the function to create training
environments with the `tune.register_env` function and connects to
the Ray cluster Azure Machine Learning service started on the GPU 
and CPU nodes. Lastly, it starts a RL training run with `tune.run()`.

We recommend setting the `local_dir` parameter to `./logs` as this
directory will automatically become available as part of the training
run's files in the Azure Portal.  The Tensorboard integration
(see "View the Tensorboard" section below) also depends on the files'
availability.  For a list of common parameter options, please refer
to the [Ray documentation](https://docs.ray.io/en/latest/rllib-training.html#common-parameters).


```python
# Taken from minecraft_environment.py and minecraft_train.py

# Define a function to create a MineRL environment
def create_env(config):
    mission = config['mission']
    port = 1000 * config.worker_index + config.vector_index
    print('*********************************************')
    print(f'* Worker {config.worker_index} creating from mission: {mission}, port {port}')
    print('*********************************************')

    if config.worker_index == 0:
        # The first environment is only used for checking the action and observation space.
        # By using a dummy environment, there's no need to spin up a Minecraft instance behind it
        # saving some CPU resources on the head node.
        return DummyEnv()

    env = EnvWrapper(mission, port)
    env = TrackingEnv(env)
    env = FrameStack(env, 2)
    
    return env


def stop(trial_id, result):
    return result["episode_reward_mean"] >= 1 \
        or result["time_total_s"] > 5 * 60 * 60


if __name__ == '__main__':
    tune.register_env("Minecraft", create_env)

    ray.init(address='auto')

    tune.run(
        run_or_experiment="IMPALA",
        config={
            "env": "Minecraft",
            "env_config": {
                "mission": "minecraft_missions/lava_maze-v0.xml"
            },
            "num_workers": 10,
            "num_cpus_per_worker": 2,
            "rollout_fragment_length": 50,
            "train_batch_size": 1024,
            "replay_buffer_num_slots": 4000,
            "replay_proportion": 10,
            "learner_queue_timeout": 900,
            "num_sgd_iter": 2,
            "num_data_loader_buffers": 2,
            "exploration_config": {
                "type": "EpsilonGreedy",
                "initial_epsilon": 1.0,
                "final_epsilon": 0.02,
                "epsilon_timesteps": 500000
            },
            "callbacks": {"on_train_result": callbacks.on_train_result},
        },
        stop=stop,
        checkpoint_at_end=True,
        local_dir='./logs'
    )
```

### Submitting a training run

Below, you create the training run using a `ReinforcementLearningEstimator`
object, which contains all the configuration parameters for this experiment:
- `source_directory`: Contains the training script and helper files to be
copied onto the node running the Ray head.
- `entry_script`: The training script, described in more detail above..
- `compute_target`: The compute target for the Ray head and training
script execution.
- `environment`: The Azure machine learning environment definition for
the node running the Ray head.
- `worker_configuration`: The configuration object for the additional
Ray nodes to be attached to the Ray cluster:
  - `compute_target`: The compute target for the additional Ray nodes.
  - `node_count`: The number of nodes to attach to the Ray cluster.
  - `environment`: The environment definition for the additional Ray nodes.
- `max_run_duration_seconds`: The time after which to abort the run if it
is still running.
- `shm_size`: The size of docker container's shared memory block. 

For more details, please take a look at the [online documentation](https://docs.microsoft.com/en-us/python/api/azureml-contrib-reinforcementlearning/?view=azure-ml-py)
for Azure Machine Learning service's reinforcement learning offering.

We configure 8 extra D2 (worker) nodes for the Ray cluster, giving us a total of
22 CPUs and 1 GPU.  The GPU and one CPU are used by the IMPALA learner,
and each MineRL environment receives 2 CPUs allowing us to spawn a total
of 10 rollout workers (see `num_workers` parameter in the training script).


Lastly, the `RunDetails` widget displays information about the submitted
RL experiment, including a link to the Azure portal with more details.

In [ ]:
from azureml.contrib.train.rl import ReinforcementLearningEstimator, WorkerConfiguration
from azureml.widgets import RunDetails

worker_config = WorkerConfiguration(
    compute_target=cpu_cluster, 
    node_count=8,
    environment=cpu_minecraft_env)

rl_est = ReinforcementLearningEstimator(
    source_directory='files',
    entry_script='minecraft_train.py',
    compute_target=gpu_cluster,
    environment=gpu_minecraft_env,
    worker_configuration=worker_config,
    max_run_duration_seconds=6 * 60 * 60,
    shm_size=1024 * 1024 * 1024 * 30)

train_run = exp.submit(rl_est)

RunDetails(train_run).show()

In [ ]:
# If you wish to cancel the run before it completes, uncomment and execute:
#train_run.cancel()

## Monitoring training progress

### View the Tensorboard

The Tensorboard can be displayed via the Azure Machine Learning service's
[Tensorboard API](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-monitor-tensorboard).
When running locally, please make sure to follow the instructions in the
link and install required packages.  Running this cell will output a URL
for the Tensorboard.

Note that the training script sets the log directory when starting RLlib
via the `local_dir` parameter.  `./logs` will automatically appear in
the downloadable files for a run.  Since this script is executed on the
Ray head node run, we need to get a reference to it as shown below.

The Tensorboard API will continuously stream logs from the run.

**Note: It may take a couple of minutes after the run is in "Running" state
before Tensorboard files are available and the board will refresh automatically**

In [ ]:
import time
from azureml.tensorboard import Tensorboard

head_run = None

timeout = 60
while timeout > 0 and head_run is None:
    timeout -= 1
    
    try:
        head_run = next(r for r in train_run.get_children() if r.id.endswith('head'))
    except StopIteration:
        time.sleep(1)

tb = Tensorboard([head_run], port=6007)
tb.start()

## Review results

Please ensure that the training run has completed before continuing with this section.

In [ ]:
train_run.wait_for_completion()

print('Training run completed.')

**Please note:** If the final "episode_reward_mean" metric from the training run is negative,
the produced model does not solve the problem of navigating the maze well.  You can view
the metric on the Tensorboard or in "Metrics" section of the head run in the Azure Machine Learning
portal.  We recommend training a new model by rerunning the notebook starting from "Submitting a training run".


### Export final model

The key result from the training run is the final checkpoint
containing the state of the IMPALA trainer (model) upon meeting the
stopping criteria specified in `minecraft_train.py`.

Azure Machine Learning service offers the [Model.register()](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.model.model?view=azure-ml-py)
API which allows you to persist the model files from the
training run.  We identify the directory containing the
final model written during the training run and register
it with Azure Machine Learning service.  We use a Dataset
object to filter out the correct files.

In [ ]:
import re
import tempfile

from azureml.core import Dataset

path_prefix = os.path.join(tempfile.gettempdir(), 'tmp_training_artifacts')

run_artifacts_path = os.path.join('azureml', head_run.id)
datastore = ws.get_default_datastore()

run_artifacts_ds = Dataset.File.from_files(datastore.path(os.path.join(run_artifacts_path, '**')))

cp_pattern = re.compile('.*checkpoint-\\d+$')

checkpoint_files = [file for file in run_artifacts_ds.to_path() if cp_pattern.match(file)]

# There should only be one checkpoint with our training settings...
final_checkpoint = os.path.dirname(os.path.join(run_artifacts_path, os.path.normpath(checkpoint_files[-1][1:])))
datastore.download(target_path=path_prefix, prefix=final_checkpoint.replace('\\', '/'), show_progress=True)

print('Download complete.')

In [ ]:
from azureml.core.model import Model

model_name = 'final_model_minecraft_maze'

model = Model.register(
    workspace=ws,
    model_path=os.path.join(path_prefix, final_checkpoint),
    model_name=model_name,
    description='Model of an agent trained to navigate a lava maze in Minecraft.')

Models can be used through a varity of APIs.  Please see the
[documentation](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-and-where)
for more details.

### Test agent performance in a rollout

To observe the trained agent's behavior, it is a common practice to
view its behavior in a rollout.  The previous reinforcement learning
tutorials explain rollouts in more detail.

The provided `minecraft_rollout.py` script loads the final checkpoint
of the trained agent from the model registered with Azure Machine Learning
service.  It then starts a rollout on 4 different lava maze layouts, that
are all larger and thus more difficult than the maze the agent was trained
on.  The script further records videos by replaying the agent's decisions
in [Malmo](https://github.com/microsoft/malmo).  Malmo supports multiple
agents in the same environment, thus allowing us to capture videos that
depict the agent from another agent's perspective.  The provided
`malmo_video_recorder.py` file and the Malmo Github repository have more
details on the video recording setup.

You can view the rewards for each rollout episode in the logs for the 'head'
run submitted below.  In some episodes, the agent may fail to reach the goal
due to the higher level of difficulty - in practice, we could continue
training the agent on harder tasks starting with the final checkpoint.

In [ ]:
script_params = {
    '--model_name': model_name
}

rollout_est = ReinforcementLearningEstimator(
    source_directory='files',
    entry_script='minecraft_rollout.py',
    script_params=script_params,
    compute_target=gpu_cluster,
    environment=gpu_minecraft_env,
    shm_size=1024 * 1024 * 1024 * 30)

rollout_run = exp.submit(rollout_est)

RunDetails(rollout_run).show()

### View videos captured during rollout

To inspect the agent's training progress you can view the videos captured
during the rollout episodes.  First, ensure that the training run has
completed.

In [ ]:
rollout_run.wait_for_completion()

head_run_rollout = next(r for r in rollout_run.get_children() if r.id.endswith('head'))

print('Rollout completed.')

Next, you need to download the video files from the training run.  We use a
Dataset to filter out the video files which are in tgz archives.

In [ ]:
rollout_run_artifacts_path = os.path.join('azureml', head_run_rollout.id)
datastore = ws.get_default_datastore()

rollout_run_artifacts_ds = Dataset.File.from_files(datastore.path(os.path.join(rollout_run_artifacts_path, '**')))

video_archives = [file for file in rollout_run_artifacts_ds.to_path() if file.endswith('.tgz')]
video_archives = [os.path.join(rollout_run_artifacts_path, os.path.normpath(file[1:])) for file in video_archives]

datastore.download(
    target_path=path_prefix,
    prefix=os.path.dirname(video_archives[0]).replace('\\', '/'),
    show_progress=True)

print('Download complete.')

Next, unzip the video files and rename them by the Minecraft mission seed used
(see `minecraft_rollout.py` for more details on how the seed is used).

In [ ]:
import tarfile
import shutil

training_artifacts_dir = './training_artifacts'
video_dir = os.path.join(training_artifacts_dir, 'videos')
video_files = []

for tar_file_path in video_archives:
    seed = tar_file_path[tar_file_path.index('rollout_') + len('rollout_'): tar_file_path.index('.tgz')]
    
    tar = tarfile.open(os.path.join(path_prefix, tar_file_path).replace('\\', '/'), 'r')
    tar_info = next(t_info for t_info in tar.getmembers() if t_info.name.endswith('mp4'))
    tar.extract(tar_info, video_dir)
    tar.close()
    
    unzipped_folder = os.path.join(video_dir, next(f_ for f_ in os.listdir(video_dir) if not f_.endswith('mp4')))    
    video_file = os.path.join(unzipped_folder,'video.mp4')
    final_video_path = os.path.join(video_dir, '{seed}.mp4'.format(seed=seed))
    
    shutil.move(video_file, final_video_path)    
    video_files.append(final_video_path)
    
    shutil.rmtree(unzipped_folder)

# Clean up any downloaded 'tmp' files
shutil.rmtree(path_prefix)

print('Local video files:\n', video_files)

Finally, run the cell below to display the videos in-line.  In some cases,
the agent may struggle to find the goal since the maze size was increased
compared to training.

In [ ]:
from IPython.core.display import display, HTML

index = 0
while index < len(video_files) - 1:
    display(
        HTML('\
        <video controls alt="cannot display video" autoplay loop width=49%> \
            <source src="{f1}" type="video/mp4"> \
        </video> \
        <video controls alt="cannot display video" autoplay loop width=49%> \
            <source src="{f2}" type="video/mp4"> \
        </video>'.format(f1=video_files[index], f2=video_files[index + 1]))
    )
    
    index += 2

if index < len(video_files):
    display(
        HTML('\
        <video controls alt="cannot display video" autoplay loop width=49%> \
            <source src="{f1}" type="video/mp4"> \
        </video>'.format(f1=video_files[index]))
    )

## Cleaning up

Below, you can find code snippets for your convenience to clean up any resources created as part of this tutorial you don't wish to retain.

In [ ]:
# to stop the Tensorboard, uncomment and run
#tb.stop()

In [ ]:
# to delete the gpu compute target, uncomment and run
#gpu_cluster.delete()

In [ ]:
# to delete the cpu compute target, uncomment and run
#cpu_cluster.delete()

In [ ]:
# to delete the registered model, uncomment and run
#model.delete()

In [ ]:
# to delete the local video files, uncomment and run
#shutil.rmtree(training_artifacts_dir)

## Next steps

This is currently the last introductory tutorial for Azure Machine Learning
service's Reinforcement
Learning offering.  We would love to hear your feedback to build the features
you need!

